In [1]:
import os
import sys
import time
import glob
import numpy as np
import torch
import utils_imagenet
import logging
import argparse
import torch.nn as nn
import genotypes
import torch.utils
import torchvision.datasets as dset
import torch.backends.cudnn as cudnn

from torch.autograd import Variable
from model import NetworkCIFAR as Network


parser = argparse.ArgumentParser("cifar")
parser.add_argument('--data', type=str, default='/home/mzhang3/Data/DARTS/darts-master/data', help='location of the data corpus')
parser.add_argument('--batch_size', type=int, default=48, help='batch size')
parser.add_argument('--learning_rate', type=float, default=0.025, help='init learning rate')
parser.add_argument('--momentum', type=float, default=0.9, help='momentum')
parser.add_argument('--weight_decay', type=float, default=3e-4, help='weight decay')
parser.add_argument('--report_freq', type=float, default=50, help='report frequency')
parser.add_argument('--gpu', type=int, default=0, help='gpu device id')
parser.add_argument('--epochs', type=int, default=600, help='num of training epochs')
parser.add_argument('--init_channels', type=int, default=36, help='num of init channels')
parser.add_argument('--layers', type=int, default=20, help='total number of layers')
parser.add_argument('--model_path', type=str, default='saved_models', help='path to save the model')
parser.add_argument('--auxiliary', action='store_true', default=True, help='use auxiliary tower')
parser.add_argument('--auxiliary_weight', type=float, default=0.4, help='weight for auxiliary loss')
parser.add_argument('--cutout', action='store_true', default=True, help='use cutout')
parser.add_argument('--cutout_length', type=int, default=16, help='cutout length')
parser.add_argument('--drop_path_prob', type=float, default=0.2, help='drop path probability')
parser.add_argument('--save', type=str, default='EXP', help='experiment name')
parser.add_argument('--seed', type=int, default=0, help='random seed')
parser.add_argument('--arch', type=str, default='ENNAS', help='which architecture to use')
parser.add_argument('--grad_clip', type=float, default=5, help='gradient clipping')

args = parser.parse_args([])
CLASSES = 10

In [2]:
np.random.seed(args.seed)
torch.cuda.set_device(args.gpu)
cudnn.benchmark = True
torch.manual_seed(args.seed)
cudnn.enabled=True
torch.cuda.manual_seed(args.seed)
logging.info('gpu device = %d' % args.gpu)
logging.info("args = %s", args)


In [3]:
genotype = eval("genotypes.%s" % args.arch)
pre_model = Network(args.init_channels, CLASSES, args.layers, args.auxiliary, genotype)
pre_model = pre_model.cuda()
utils_imagenet.load(pre_model, 'cifar10_600.pt')

108 108 36
108 144 36
144 144 36
144 144 36
144 144 36
144 144 36
144 144 72
144 288 72
288 288 72
288 288 72
288 288 72
288 288 72
288 288 72
288 288 144
288 576 144
576 576 144
576 576 144
576 576 144
576 576 144
576 576 144


In [4]:
pre_model.load_state_dict(torch.load('cifar10_600.pt'))

<All keys matched successfully>

In [5]:
import torch
import torchvision
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torch.nn.functional as F
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset
import os
from PIL import Image
import matplotlib.pyplot as plt
from torch.optim.lr_scheduler import StepLR
import numpy as np
from PIL import ImageFile
from torch.utils.tensorboard import SummaryWriter
from datetime import datetime
import numpy as np
import pandas as pd
import os
import random 
from shutil import copyfile
from torch.utils.data import Dataset
from torchvision.datasets import ImageFolder
from PIL import Image
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
import re
import albumentations as albu
from albumentations.pytorch import ToTensor
from catalyst.data import Augmentor
import torchxrayvision as xrv


import torch
import torchvision
from torchvision import transforms, utils
from torch.utils.data import Dataset, DataLoader
import matplotlib.pyplot as plt
import torch.optim as optim
from torch.utils.data import Dataset
import os
from PIL import Image
import matplotlib.pyplot as plt
from torch.optim.lr_scheduler import StepLR
from PIL import Image
import torch.nn.functional as F
import torch.nn as nn
import numpy as np
from sklearn.metrics import roc_auc_score
import re
import albumentations as albu
from albumentations.pytorch import ToTensor
from catalyst.data import Augmentor
from skimage.io import imread, imsave
import skimage

/data/taoliu/anaconda3/lib/python3.6/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:541: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/data/taoliu/anaconda3/lib/python3.6/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:542: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/data/taoliu/anaconda3/lib/python3.6/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:543: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/data/taoliu/anaconda3/lib/python3.6/site-packages/tensorboar

In [6]:
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
train_transformer = transforms.Compose([
    transforms.Resize(48),
    transforms.RandomResizedCrop((32),scale=(0.5,1.0)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    normalize
])

val_transformer = transforms.Compose([
    transforms.Resize(32),
    transforms.CenterCrop(32),
    transforms.ToTensor(),
    normalize
])


# In[80]:


batchsize=4
def read_txt(txt_path):
    with open(txt_path) as f:
        lines = f.readlines()
    txt_data = [line.strip() for line in lines]
    return txt_data

class CovidCTDataset(Dataset):
    def __init__(self, root_dir, txt_COVID, txt_NonCOVID, transform=None):
        """
        Args:
            txt_path (string): Path to the txt file with annotations.
            root_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        File structure:
        - root_dir
            - CT_COVID
                - img1.png
                - img2.png
                - ......
            - CT_NonCOVID
                - img1.png
                - img2.png
                - ......
        """
        self.root_dir = root_dir
        self.txt_path = [txt_COVID,txt_NonCOVID]
        self.classes = ['CT_COVID', 'CT_NonCOVID']
        self.num_cls = len(self.classes)
        self.img_list = []
        for c in range(self.num_cls):
            cls_list = [[os.path.join(self.root_dir,self.classes[c],item), c] for item in read_txt(self.txt_path[c])]
            self.img_list += cls_list
        self.transform = transform

    def __len__(self):
        return len(self.img_list)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        img_path = self.img_list[idx][0]
        image = Image.open(img_path).convert('RGB')

        if self.transform:
            image = self.transform(image)
        sample = {'img': image,
                  'label': int(self.img_list[idx][1])}
        return sample


In [7]:
trainset = CovidCTDataset(root_dir='/tmp/mozilla_taoliu0/COVID-CT-master/Images-processed',
                          txt_COVID='/tmp/mozilla_taoliu0/COVID-CT-master/Data-split/COVID/trainCT_COVID.txt',
                          txt_NonCOVID='/tmp/mozilla_taoliu0/COVID-CT-master/Data-split/NonCOVID/trainCT_NonCOVID.txt',
                          transform= train_transformer)
valset = CovidCTDataset(root_dir='/tmp/mozilla_taoliu0/COVID-CT-master/Images-processed',
                          txt_COVID='/tmp/mozilla_taoliu0/COVID-CT-master/Data-split/COVID/valCT_COVID.txt',
                          txt_NonCOVID='/tmp/mozilla_taoliu0/COVID-CT-master/Data-split/NonCOVID/valCT_NonCOVID.txt',
                          transform= val_transformer)
testset = CovidCTDataset(root_dir='/tmp/mozilla_taoliu0/COVID-CT-master/Images-processed',
                          txt_COVID='/tmp/mozilla_taoliu0/COVID-CT-master/Data-split/COVID/testCT_COVID.txt',
                          txt_NonCOVID='/tmp/mozilla_taoliu0/COVID-CT-master/Data-split/NonCOVID/testCT_NonCOVID.txt',
                          transform= val_transformer)
print(trainset.__len__())
print(valset.__len__())
print(testset.__len__())

train_loader = DataLoader(trainset, batch_size=batchsize, drop_last=True, shuffle=True,pin_memory=True, num_workers=4)
val_loader = DataLoader(valset, batch_size=batchsize, drop_last=True, shuffle=False,pin_memory=True, num_workers=4)
test_loader = DataLoader(testset, batch_size=batchsize, drop_last=True, shuffle=False,pin_memory=True, num_workers=4)

424
116
200


In [8]:
modelname = 'NAS_CIFAR_Model'

In [9]:
alpha = None
device = 'cuda'

model = Network(args.init_channels, 2, args.layers, args.auxiliary, genotype)
model = torch.nn.DataParallel(model)
new_model = model.cuda()
model.to(device)

108 108 36
108 144 36
144 144 36
144 144 36
144 144 36
144 144 36
144 144 72
144 288 72
288 288 72
288 288 72
288 288 72
288 288 72
288 288 72
288 288 144
288 576 144
576 576 144
576 576 144
576 576 144
576 576 144
576 576 144


DataParallel(
  (module): NetworkCIFAR(
    (stem): Sequential(
      (0): Conv2d(3, 108, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (1): BatchNorm2d(108, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (cells): ModuleList(
      (0): Cell(
        (preprocess0): ReLUConvBN(
          (op): Sequential(
            (0): ReLU()
            (1): Conv2d(108, 36, kernel_size=(1, 1), stride=(1, 1), bias=False)
            (2): BatchNorm2d(36, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          )
        )
        (preprocess1): ReLUConvBN(
          (op): Sequential(
            (0): ReLU()
            (1): Conv2d(108, 36, kernel_size=(1, 1), stride=(1, 1), bias=False)
            (2): BatchNorm2d(36, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          )
        )
        (_ops): ModuleList(
          (0): SepConv(
            (op): Sequential(
              (0): ReLU()
              (1): Conv2d(3

In [10]:
pre_model = torch.nn.DataParallel(pre_model)

newmodel_dict=model.state_dict()
premodel_dict=pre_model.state_dict()
new_list=list(newmodel_dict.keys())
pre_list=list(premodel_dict.keys())
for i in range(1352):####The model contains 1356 keys, we need exclude the last layer
    newmodel_dict[new_list[i]]=premodel_dict[pre_list[i]]

model.load_state_dict(newmodel_dict)

model=model.module
#dict_name=list(model.state_dict())
#for i,p in enumerate(dict_name):
  #  print(i,p)

In [11]:

def train(train_loader, model, criterion, optimizer):
    objs = utils_imagenet.AvgrageMeter()
    top1 = utils_imagenet.AvgrageMeter()
    model.train()
    
    loss = 0
    train_correct = 0

    for step, batch_samples in enumerate(train_loader):
        
        input, target = batch_samples['img'].to(device), batch_samples['label'].to(device)
      #  input = input[:, 0, :, :]
      #  input = input[:, None, :, :]
#         data, targets_a, targets_b, lam = mixup_data(data, target.long(), alpha, use_cuda=True)
      #  input = input.repeat(1,3,1,1)        
        
        
        target = target.cuda(async=True)
        input = input.cuda()
        input = Variable(input)
        target = Variable(target)

        optimizer.zero_grad()
        logits, logits_aux = model(input)
        loss = criterion(logits, target)
        if args.auxiliary:
            loss_aux = criterion(logits_aux, target)
            loss += args.auxiliary_weight*loss_aux

        loss.backward()
        nn.utils.clip_grad_norm(model.parameters(), args.grad_clip)
        optimizer.step()

        prec1, prec2 = utils_imagenet.accuracy(logits, target, topk=(1,2))
        n = input.size(0)
        objs.update(loss.data, n)
        top1.update(prec1.data, n)
       # top2.update(prec5.data, n)

        if step % args.report_freq == 0:
            logging.info('train %03d %e %f', step, objs.avg, top1.avg)
            


        pred = logits.argmax(dim=1, keepdim=True)
        train_correct += pred.eq(target.long().view_as(pred)).sum().item()
    
        # Display progress and write to tensorboard
        if step % bs == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tTrain Loss: {:.6f}'.format(
                epoch, step, len(train_loader),
                100.0 * step / len(train_loader), loss.item()/ bs))
    
    print('\nTrain set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        loss/len(train_loader.dataset), train_correct, len(train_loader.dataset),
        100.0 * train_correct / len(train_loader.dataset)))
    f = open('model_result/{}.txt'.format(modelname), 'a+')
    f.write('\nTrain set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        loss/len(train_loader.dataset), train_correct, len(train_loader.dataset),
        100.0 * train_correct / len(train_loader.dataset)))
    f.write('\n')
    f.close()
            
            

    return top1.avg, objs.avg    
    
    
def infer(val_loader, model, criterion,epoch):
    
    objs = utils_imagenet.AvgrageMeter()
    top1 = utils_imagenet.AvgrageMeter()
    top5 = utils_imagenet.AvgrageMeter()
    
    model.eval()
    test_loss = 0
    correct = 0
    results = []
    
    TP = 0
    TN = 0
    FN = 0
    FP = 0
    
    
    # Don't update model
    with torch.no_grad():
        tpr_list = []
        fpr_list = []
        
        predlist=[]
        scorelist=[]
        targetlist=[]
        # Predict
        
    
        for batch_index, batch_samples in enumerate(val_loader):
            data, target = batch_samples['img'].to(device), batch_samples['label'].to(device)
            data = data[:, 0, :, :]
            data = data[:, None, :, :]
            data = data.repeat(1,3,1,1)
            data = Variable(data, volatile=True).cuda()
            target = Variable(target, volatile=True).cuda(async=True)            
            
            
            output, logits_aux = model(data)
            
            test_loss += criterion(output, target)
            score = F.softmax(output, dim=1)
            pred = output.argmax(dim=1, keepdim=True)
#             print('target',target.long()[:, 2].view_as(pred))
            correct += pred.eq(target.long().view_as(pred)).sum().item()
            
#             print(output[:,1].cpu().numpy())
#             print((output[:,1]+output[:,0]).cpu().numpy())
#             predcpu=(output[:,1].cpu().numpy())/((output[:,1]+output[:,0]).cpu().numpy())
            targetcpu=target.long().cpu().numpy()
            predlist=np.append(predlist, pred.cpu().numpy())
            scorelist=np.append(scorelist, score.cpu().numpy()[:,1])
            targetlist=np.append(targetlist,targetcpu)
           
          
    return targetlist, scorelist, predlist





    

bs = 10
votenum = 10
import warnings
warnings.filterwarnings('ignore')

r_list = []
p_list = []
acc_list = []
AUC_list = []
# TP = 0
# TN = 0
# FN = 0
# FP = 0
#vote_pred = np.zeros(valset.__len__())#####since we dropped several data
#vote_score = np.zeros(valset.__len__())

vote_pred = np.zeros(valset.__len__())
vote_score = np.zeros(valset.__len__())




#optimizer = optim.SGD(model.parameters(), lr=0.001, momentum = 0.9)
optimizer = optim.Adam(model.parameters(), lr=0.0001)
scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=10)
#scheduler = StepLR(optimizer, step_size=1)

total_epoch = 3000
criterion = nn.CrossEntropyLoss()
criterion = criterion.cuda()

<input>:19: DeprecationWarning:

'async' and 'await' will become reserved keywords in Python 3.7

<ipython-input-11-731ad0c3d451>:19: DeprecationWarning:

'async' and 'await' will become reserved keywords in Python 3.7



In [12]:
for epoch in range(total_epoch):

    scheduler.step()
    model.drop_path_prob = args.drop_path_prob * epoch / args.epochs
    train_acc, train_obj = train(train_loader, model, criterion, optimizer)
    targetlist, scorelist, predlist = infer(val_loader, model, criterion,epoch)
    
    print('target',targetlist)
    print('score',scorelist)
    print('predict',predlist)
    vote_pred = vote_pred + predlist 
    vote_score = vote_score + scorelist 

    if epoch % votenum == 0:
        
        # major vote
        vote_pred[vote_pred <= (votenum/2)] = 0
        vote_pred[vote_pred > (votenum/2)] = 1
        vote_score = vote_score/votenum
        
        print('vote_pred', vote_pred)
        print('targetlist', targetlist)
        TP = ((vote_pred == 1) & (targetlist == 1)).sum()
        TN = ((vote_pred == 0) & (targetlist == 0)).sum()
        FN = ((vote_pred == 0) & (targetlist == 1)).sum()
        FP = ((vote_pred == 1) & (targetlist == 0)).sum()
        
        
        print('TP=',TP,'TN=',TN,'FN=',FN,'FP=',FP)
        print('TP+FP',TP+FP)
        p = TP / (TP + FP)
        print('precision',p)
        p = TP / (TP + FP)
        r = TP / (TP + FN)
        print('recall',r)
        F1 = 2 * r * p / (r + p)
        acc = (TP + TN) / (TP + TN + FP + FN)
        print('F1',F1)
        print('acc',acc)
        AUC = roc_auc_score(targetlist, vote_score)
        print('AUCp', roc_auc_score(targetlist, vote_pred))
        print('AUC', AUC)
        
        
        
#         if epoch == total_epoch:
        torch.save(model.state_dict(), "model_backup/{}.pt".format(modelname))  
        
        vote_pred = np.zeros(valset.__len__())
        vote_score = np.zeros(valset.__len__())
        print('\n The epoch is {}, average recall: {:.4f}, average precision: {:.4f},average F1: {:.4f}, average accuracy: {:.4f}, average AUC: {:.4f}'.format(
        epoch, r, p, F1, acc, AUC))

        f = open('model_result/{}.txt'.format(modelname), 'a+')
        f.write('\n The epoch is {}, average recall: {:.4f}, average precision: {:.4f},average F1: {:.4f}, average accuracy: {:.4f}, average AUC: {:.4f}'.format(
        epoch, r, p, F1, acc, AUC))
        f.close()    
    

Train Epoch: 0 [0/106 (0%)]	Train Loss: 0.093001
Train Epoch: 0 [10/106 (9%)]	Train Loss: 0.098486
Train Epoch: 0 [20/106 (19%)]	Train Loss: 0.074945
Train Epoch: 0 [30/106 (28%)]	Train Loss: 0.092415
Train Epoch: 0 [40/106 (38%)]	Train Loss: 0.072195
Train Epoch: 0 [50/106 (47%)]	Train Loss: 0.071060
Train Epoch: 0 [60/106 (57%)]	Train Loss: 0.099106
Train Epoch: 0 [70/106 (66%)]	Train Loss: 0.086640
Train Epoch: 0 [80/106 (75%)]	Train Loss: 0.092945
Train Epoch: 0 [90/106 (85%)]	Train Loss: 0.113790
Train Epoch: 0 [100/106 (94%)]	Train Loss: 0.067097

Train set: Average loss: 0.0024, Accuracy: 282/424 (67%)

target [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
score [0.26936156 0.2

Train Epoch: 3 [0/106 (0%)]	Train Loss: 0.095509
Train Epoch: 3 [10/106 (9%)]	Train Loss: 0.161592
Train Epoch: 3 [20/106 (19%)]	Train Loss: 0.043424
Train Epoch: 3 [30/106 (28%)]	Train Loss: 0.129203
Train Epoch: 3 [40/106 (38%)]	Train Loss: 0.026931
Train Epoch: 3 [50/106 (47%)]	Train Loss: 0.044906
Train Epoch: 3 [60/106 (57%)]	Train Loss: 0.050206
Train Epoch: 3 [70/106 (66%)]	Train Loss: 0.082228
Train Epoch: 3 [80/106 (75%)]	Train Loss: 0.021845
Train Epoch: 3 [90/106 (85%)]	Train Loss: 0.122352
Train Epoch: 3 [100/106 (94%)]	Train Loss: 0.015238

Train set: Average loss: 0.0007, Accuracy: 339/424 (80%)

target [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
score [2.38562059e-02

Train Epoch: 6 [0/106 (0%)]	Train Loss: 0.012671
Train Epoch: 6 [10/106 (9%)]	Train Loss: 0.015499
Train Epoch: 6 [20/106 (19%)]	Train Loss: 0.018147
Train Epoch: 6 [30/106 (28%)]	Train Loss: 0.023679
Train Epoch: 6 [40/106 (38%)]	Train Loss: 0.035353
Train Epoch: 6 [50/106 (47%)]	Train Loss: 0.032411
Train Epoch: 6 [60/106 (57%)]	Train Loss: 0.020572
Train Epoch: 6 [70/106 (66%)]	Train Loss: 0.043394
Train Epoch: 6 [80/106 (75%)]	Train Loss: 0.024900
Train Epoch: 6 [90/106 (85%)]	Train Loss: 0.187018
Train Epoch: 6 [100/106 (94%)]	Train Loss: 0.020190

Train set: Average loss: 0.0003, Accuracy: 372/424 (88%)

target [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
score [0.02052706 0.0

Train Epoch: 9 [0/106 (0%)]	Train Loss: 0.150686
Train Epoch: 9 [10/106 (9%)]	Train Loss: 0.152149
Train Epoch: 9 [20/106 (19%)]	Train Loss: 0.025796
Train Epoch: 9 [30/106 (28%)]	Train Loss: 0.007444
Train Epoch: 9 [40/106 (38%)]	Train Loss: 0.023224
Train Epoch: 9 [50/106 (47%)]	Train Loss: 0.009139
Train Epoch: 9 [60/106 (57%)]	Train Loss: 0.021919
Train Epoch: 9 [70/106 (66%)]	Train Loss: 0.013991
Train Epoch: 9 [80/106 (75%)]	Train Loss: 0.006521
Train Epoch: 9 [90/106 (85%)]	Train Loss: 0.007248
Train Epoch: 9 [100/106 (94%)]	Train Loss: 0.018593

Train set: Average loss: 0.0035, Accuracy: 373/424 (88%)

target [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
score [1.79718807e-02

Train Epoch: 12 [0/106 (0%)]	Train Loss: 0.014706
Train Epoch: 12 [10/106 (9%)]	Train Loss: 0.011806
Train Epoch: 12 [20/106 (19%)]	Train Loss: 0.040651
Train Epoch: 12 [30/106 (28%)]	Train Loss: 0.136017
Train Epoch: 12 [40/106 (38%)]	Train Loss: 0.014514
Train Epoch: 12 [50/106 (47%)]	Train Loss: 0.048878
Train Epoch: 12 [60/106 (57%)]	Train Loss: 0.015711
Train Epoch: 12 [70/106 (66%)]	Train Loss: 0.042907
Train Epoch: 12 [80/106 (75%)]	Train Loss: 0.029608
Train Epoch: 12 [90/106 (85%)]	Train Loss: 0.005996
Train Epoch: 12 [100/106 (94%)]	Train Loss: 0.011243

Train set: Average loss: 0.0017, Accuracy: 378/424 (89%)

target [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
score [1.0

...

Train Epoch: 207 [0/106 (0%)]	Train Loss: 0.004227
Train Epoch: 207 [10/106 (9%)]	Train Loss: 0.000293
Train Epoch: 207 [20/106 (19%)]	Train Loss: 0.000060
Train Epoch: 207 [30/106 (28%)]	Train Loss: 0.000175
Train Epoch: 207 [40/106 (38%)]	Train Loss: 0.042777
Train Epoch: 207 [50/106 (47%)]	Train Loss: 0.000020
Train Epoch: 207 [60/106 (57%)]	Train Loss: 0.000026
Train Epoch: 207 [70/106 (66%)]	Train Loss: 0.000055
Train Epoch: 207 [80/106 (75%)]	Train Loss: 0.000291
Train Epoch: 207 [90/106 (85%)]	Train Loss: 0.000013
Train Epoch: 207 [100/106 (94%)]	Train Loss: 0.000019

Train set: Average loss: 0.0000, Accuracy: 421/424 (99%)

target [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]

Train Epoch: 210 [0/106 (0%)]	Train Loss: 0.000040
Train Epoch: 210 [10/106 (9%)]	Train Loss: 0.133727
Train Epoch: 210 [20/106 (19%)]	Train Loss: 0.000024
Train Epoch: 210 [30/106 (28%)]	Train Loss: 0.105691
Train Epoch: 210 [40/106 (38%)]	Train Loss: 0.000016
Train Epoch: 210 [50/106 (47%)]	Train Loss: 0.000127
Train Epoch: 210 [60/106 (57%)]	Train Loss: 0.000269
Train Epoch: 210 [70/106 (66%)]	Train Loss: 0.000003
Train Epoch: 210 [80/106 (75%)]	Train Loss: 0.000065
Train Epoch: 210 [90/106 (85%)]	Train Loss: 0.000045
Train Epoch: 210 [100/106 (94%)]	Train Loss: 0.000011

Train set: Average loss: 0.0000, Accuracy: 417/424 (98%)

target [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/data/taoliu/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py", line 2963, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-12-efee3a4b46be>", line 5, in <module>
    train_acc, train_obj = train(train_loader, model, criterion, optimizer)
  File "<ipython-input-11-731ad0c3d451>", line 33, in train
    optimizer.step()
  File "/data/taoliu/anaconda3/lib/python3.6/site-packages/torch/optim/lr_scheduler.py", line 66, in wrapper
    return wrapped(*args, **kwargs)
  File "/data/taoliu/anaconda3/lib/python3.6/site-packages/torch/optim/adam.py", line 103, in step
    denom = (exp_avg_sq.sqrt() / math.sqrt(bias_correction2)).add_(group['eps'])
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/data/taoliu/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py", line 1863, in showtraceback
    s

KeyboardInterrupt: 